In [9]:
import pandas as pd
import numpy as np
from pathlib import Path

In [10]:
# 1) Load raw CSV as strings (avoids locale/type traps)
from pathlib import Path

src_path = Path("C:/Users/Hasya-ZanrooMY/Downloads/Crimes_-_2001_to_Present_20251220.csv")
df = pd.read_csv(src_path, dtype=str, low_memory=False)

In [11]:
# 2) Robust datetime parser (US month-first + day-first fallback)
def parse_mixed_datetime(series: pd.Series) -> pd.Series:
    s = series.astype("string").str.strip()
    dt = pd.to_datetime(s, errors="coerce", dayfirst=False)     # US first
    mask = dt.isna() & s.notna() & (s != "")
    dt.loc[mask] = pd.to_datetime(s[mask], errors="coerce", dayfirst=True)  # fallback
    return dt

In [12]:
#3) Parse datetimes
df["date_time"] = parse_mixed_datetime(df["Date"])
df["updated_on_dt"] = parse_mixed_datetime(df["Updated On"])

C:\Users\Hasya-ZanrooMY\AppData\Local\Temp\ipykernel_2852\2063919168.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(s, errors="coerce", dayfirst=False)     # US first
C:\Users\Hasya-ZanrooMY\AppData\Local\Temp\ipykernel_2852\2063919168.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(s, errors="coerce", dayfirst=False)     # US first


In [13]:
# 4) Split Date & Time
df["date_only"] = df["date_time"].dt.date
df["time_only"] = df["date_time"].dt.time

In [14]:
# 5) Add dashboard helper columns
df["year"] = df["date_time"].dt.year.astype("Int64")
df["month"] = df["date_time"].dt.month.astype("Int64")
df["month_name"] = df["date_time"].dt.month_name()
df["day_of_week"] = df["date_time"].dt.day_name()
df["hour"] = df["date_time"].dt.hour.astype("Int64")
df["week"] = df["date_time"].dt.isocalendar().week.astype("Int64")

In [15]:
# 6) Normalize booleans
def to_bool(x):
    if pd.isna(x): return pd.NA
    x = str(x).strip().lower()
    if x in {"true","t","yes","y","1"}: return True
    if x in {"false","f","no","n","0"}: return False
    return pd.NA

    
df["Arrest"] = df["Arrest"].map(to_bool).astype("boolean")
df["Domestic"] = df["Domestic"].map(to_bool).astype("boolean")

In [16]:

# 7) Convert numeric fields safely
num_cols = ["Beat","District","Ward","Community Area","X Coordinate","Y Coordinate","Latitude","Longitude","Year"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [17]:
# 8) Standardize column names (Power BI friendly)
df.columns = (
    df.columns.str.strip().str.lower()
      .str.replace(r"[^\w]+", "_", regex=True)
      .str.strip("_")
)

In [ ]:
# 9) Remove rows with unparsed datetime (should be near 0)
df = df.dropna(subset=["date_time"])


In [18]:
# 10) Save cleaned file
out_csv = Path("Crimes_CLEAN_python.csv")
df.to_csv(out_csv, index=False)

print("Saved:", out_csv, "Rows:", len(df), "Cols:", len(df.columns))

Saved: Crimes_CLEAN_python.csv Rows: 481327 Cols: 32
